In [0]:
# Databricks notebook source
dbutils.widgets.dropdown("network", "devnet", ["devnet", "mainnet"], "Network")
# NETWORK = dbutils.widgets.get("network")
NETWORK = dbutils.widgets.get("pipeline.network")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-670709948429027> in <module> 
 2 dbutils . widgets . dropdown ( "network" , "devnet" , [ "devnet" , "mainnet" ] , "Network" ) 
 3 # NETWORK = dbutils.widgets.get("network") 
 ----> 4 NETWORK = dbutils . widgets . get ( "pipeline.network" ) 

 /databricks/python_shell/dbruntime/WidgetHandlerImpl.py in get (self, name) 
 40 : return : Current value of the widget or default value
 41 """
 ---> 42 return self . _notebookArguments . getArgument ( name , self . _entry_point . getCurrentBindings ( ) ) 
 43 
 44 def getArgument ( self , name , defaultValue = None ) : 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o323.getArgument.
: com.databricks.dbutils_v1.InputWidgetNotDefined: No input widget named pipeline.network is defined
	at com.databricks.backend.daemon.driver.NotebookArguments.checkExists(NotebookArguments.scala:72)
	at com.databricks.backend.daemon.driver.NotebookArguments.getArgument(NotebookArguments.scala:258)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [0]:
from databricks import feature_store
from databricks.feature_store.online_store_spec import AmazonDynamoDBSpec
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pytz import timezone
from datetime import datetime, timezone

In [0]:
# Don't need to run these again since they're once-off FS definition creations

# fs.create_table(
#     name='zetadex_feature_store.agg_prices_market_1h',
#     primary_keys=["timestamp"],
#     df=agg_trades_24h_rolling_df,
#     partition_columns="date_",
#     description="Rolling 24hr trade summary metrics",
# )

In [0]:
current_date = str(datetime.now(timezone.utc).date())
current_hour = datetime.now(timezone.utc).hour

In [0]:
fs = feature_store.FeatureStoreClient()

online_store = AmazonDynamoDBSpec(
  region='ap-southeast-1',
)

underlyings = ['SOL', 'BTC']

2022/07/20 14:40:26 INFO databricks.feature_store.online_store_spec.amazon_dynamodb_online_store_spec: No explicit credentials for write provided. Instance profile attached to the cluster will be used.

# Feature Store Update Routines

In [0]:
# fs.register_table(
#   delta_table="zetadex_mainnet.agg_pnl",
#   primary_keys='owner_pub_key',
#   description='Zeta DEX Account PNLs'
# )

fs.publish_table(
  name='zetadex_mainnet.agg_pnl',
  online_store=online_store,
  filter_condition=f"date_ = '{current_date}' and hour_ = '{current_hour}'",
  features=[
    'timestamp',
    'owner_pub_key',
    'pnl',
    'pnl_diff_24h',
    'pnl_diff_7d',
    'pnl_diff_30d',
    'pnl_ratio_24h',
    'pnl_ratio_7d',
    'pnl_ratio_30d',
    'pnl_ratio_24h_rank',
    'pnl_ratio_7d_rank',
    'pnl_ratio_30d_rank',
    'pnl_diff_24h_rank',
    'pnl_diff_7d_rank',
    'pnl_diff_30d_rank'],
  mode='merge'
)

2022/06/10 09:47:42 INFO databricks.feature_store.online_store_spec.amazon_dynamodb_online_store_spec: No explicit credentials for write provided. Instance profile attached to the cluster will be used.

--------------------------------------------------------------------------- 
 Exception Traceback (most recent call last)
 <command-3702164534206901> in <module> 
 3 )
 4 
 ----> 5 fs.publish_table(
 6 name = table_name , 
 7 online_store = online_store , 

 /databricks/.python_edge_libs/databricks/feature_store/client.py in publish_table (self, name, online_store, filter_condition, mode, streaming, checkpoint_location, trigger, features) 
 1692 
 1693 # Check: feature tables exists in the catalog 
 -> 1694 if not self . _catalog_client . feature_table_exists ( name , req_context ) : 
 1695 raise ValueError ( f"Feature table '{name}' does not exists in the catalog." ) 
 1696 

 /databricks/.python_edge_libs/databricks/feature_store/catalog_client.py in feature_table_exists (self, name, req_context) 
 172 if get_error_code ( e ) == ErrorCode . Name ( RESOURCE_DOES_NOT_EXIST ) : 
 173 return False 
 --> 174 raise e
 175 return True 
 176 

 /databricks/.python_edge_libs/databricks/feature_store/catalog_client.py in feature_table_exists (self, name, req_context) 
 168 """
 169 try : 
 --> 170 self . get_feature_table ( name , req_context ) 
 171 except Exception as e : 
 172 if get_error_code ( e ) == ErrorCode . Name ( RESOURCE_DOES_NOT_EXIST ) : 

 /databricks/.python_edge_libs/databricks/feature_store/catalog_client.py in get_feature_table (self, feature_table, req_context) 
 157 
 158 def get_feature_table ( self , feature_table : str , req_context : RequestContext ) : 
 --> 159 response_proto = self . _get_feature_table ( feature_table , req_context ) 
 160 return FeatureTable . from_proto ( response_proto . feature_table ) 
 161 

 /databricks/.python_edge_libs/databricks/feature_store/catalog_client.py in _get_feature_table (self, feature_table, req_context) 
 132 def _get_feature_table ( self , feature_table : str , req_context : RequestContext ) : 
 133 req_body = GetFeatureTable ( name = feature_table ) 
 --> 134 return self . _call_endpoint ( GetFeatureTable , req_body , req_context ) 
 135 
 136 # CRUD API to call Feature Catalog 

 /databricks/.python_edge_libs/databricks/feature_store/catalog_client.py in _call_endpoint (self, api, json_body, req_context) 
 121 endpoint , method = _METHOD_TO_INFO [ api ] 
 122 response_proto = api . Response ( ) 
 --> 123 return call_endpoint(
 124 self . _get_host_creds ( ) , 
 125 endpoint , 

 /databricks/.python_edge_libs/databricks/feature_store/utils/rest_utils.py in call_endpoint (host_creds, endpoint, method, json_body, response_proto, req_context) 
 199 extra_headers = req_context . get_headers ( ) , 
 200 )
 --> 201 response = verify_rest_response ( response , endpoint ) 
 202 js_dict = json . loads ( response . text ) 
 203 json_to_proto ( js_dict = js_dict , message = response_proto ) 

 /databricks/.python_edge_libs/databricks/feature_store/utils/rest_utils.py in verify_rest_response (response, endpoint) 
 118 if _can_parse_as_json ( response . text ) : 
 119 # ToDo(ML-20622): return cleaner error to client, eg: mlflow.exceptions.RestException 
 --> 120 raise Exception ( json . loads ( response . text ) ) 
 121 else : 
 122 base_msg = (

 Exception : {'error_code': 'PERMISSION_DENIED', 'message': "Request failed access control checks. User has no permissions on 'zetadex_mainnet.agg_pnl' but requires one of 'CAN_VIEW_METADATA', 'CAN_EDIT_METADATA', 'CAN_MANAGE'."}

In [0]:
# On the fly transformation to get the last 24hrs volume from our 1h agg'ed tables
agg_trades_24h_rolling_df = \
    (spark.table("zetadex_mainnet.agg_trades_1h")
     .filter(F.col("timestamp_window.start") >= (F.date_trunc("hour", F.current_timestamp()) - F.expr('INTERVAL 24 HOUR')))
     .agg(
         F.sum("trades_count").alias("trades_count"),
         F.sum("volume").alias("volume"),
         F.sum("notional_volume").alias("notional_volume"),
         F.sum("premium_sum").alias("premium_sum"),
      )
      .withColumn("timestamp", F.date_trunc("hour", F.current_timestamp()))
      .withColumn("date_", F.to_date("timestamp"))
      .withColumn("hour_", F.date_format("timestamp", "HH").cast("int"))
     )
agg_trades_24h_rolling_df.show()
# Write new results to table
fs.write_table(
  name='zetadex_feature_store.agg_trades_24h_rolling',
  df=agg_trades_24h_rolling_df,
  mode="merge",
)

fs.publish_table(
  name='zetadex_feature_store.agg_trades_24h_rolling',
  online_store=online_store,
#   filter_condition=f"date_ = '{current_date}' and hour_ = '{current_hour}'",
  features=[
    'trades_count',
    'volume',
    'notional_volume',
    'premium_sum',
    'timestamp'],
  mode='merge'
)

+------------+--------+---------------+-------------+-------------------+----------+-----+
trades_count| volume|notional_volume| premium_sum| timestamp| date_|hour_|
+------------+--------+---------------+-------------+-------------------+----------+-----+
 32|9414.632| 276234.77128|17937.9264968|2022-06-19 11:00:00|2022-06-19| 11|
+------------+--------+---------------+-------------+-------------------+----------+-----+

--------------------------------------------------------------------------- 
 Exception Traceback (most recent call last)
 <command-880748253764033> in <module> 
 15 agg_trades_24h_rolling_df . show ( ) 
 16 # Write new results to table 
 ---> 17 fs.write_table(
 18 name = 'zetadex_feature_store.agg_trades_24h_rolling' , 
 19 df = agg_trades_24h_rolling_df , 

 /databricks/.python_edge_libs/databricks/feature_store/client.py in write_table (self, name, df, mode, checkpoint_location, trigger) 
 1563 : return : If ` ` df . isStreaming ` ` , returns a PySpark : class : ` StreamingQuery < pyspark . sql . streaming . StreamingQuery > ` . : obj : ` None ` otherwise . 
 1564 """
 -> 1565 return self._write_table(
 1566 name = name , 
 1567 df = df , 

 /databricks/.python_edge_libs/databricks/feature_store/client.py in _write_table (self, name, df, mode, req_context, checkpoint_location, trigger, producer_action) 
 1362 FeatureStoreClient . _check_schema_top_level_types_supported ( df . schema ) 
 1363 self . _check_feature_table_exists_in_hive ( name ) 
 -> 1364 feature_table = self . _catalog_client . get_feature_table ( name , req_context ) 
 1365 
 1366 # We know from the successful `get_feature_table call` above that the user has 

 /databricks/.python_edge_libs/databricks/feature_store/catalog_client.py in get_feature_table (self, feature_table, req_context) 
 157 
 158 def get_feature_table ( self , feature_table : str , req_context : RequestContext ) : 
 --> 159 response_proto = self . _get_feature_table ( feature_table , req_context ) 
 160 return FeatureTable . from_proto ( response_proto . feature_table ) 
 161 

 /databricks/.python_edge_libs/databricks/feature_store/catalog_client.py in _get_feature_table (self, feature_table, req_context) 
 132 def _get_feature_table ( self , feature_table : str , req_context : RequestContext ) : 
 133 req_body = GetFeatureTable ( name = feature_table ) 
 --> 134 return self . _call_endpoint ( GetFeatureTable , req_body , req_context ) 
 135 
 136 # CRUD API to call Feature Catalog 

 /databricks/.python_edge_libs/databricks/feature_store/catalog_client.py in _call_endpoint (self, api, json_body, req_context) 
 121 endpoint , method = _METHOD_TO_INFO [ api ] 
 122 response_proto = api . Response ( ) 
 --> 123 return call_endpoint(
 124 self . _get_host_creds ( ) , 
 125 endpoint , 

 /databricks/.python_edge_libs/databricks/feature_store/utils/rest_utils.py in call_endpoint (host_creds, endpoint, method, json_body, response_proto, req_context) 
 199 extra_headers = req_context . get_headers ( ) , 
 200 )
 --> 201 response = verify_rest_response ( response , endpoint ) 
 202 js_dict = json . loads ( response . text ) 
 203 json_to_proto ( js_dict = js_dict , message = response_proto ) 

 /databricks/.python_edge_libs/databricks/feature_store/utils/rest_utils.py in verify_rest_response (response, endpoint) 
 118 if _can_parse_as_json ( response . text ) : 
 119 # ToDo(ML-20622): return cleaner error to client, eg: mlflow.exceptions.RestException 
 --> 120 raise Exception ( json . loads ( response . text ) ) 
 121 else : 
 122 base_msg = (

 Exception : {'error_code': 'PERMISSION_DENIED', 'message': "Request failed access control checks. User has no permissions on 'zetadex_feature_store.agg_trades_24h_rolling' but requires one of 'CAN_VIEW_METADATA', 'CAN_EDIT_METADATA', 'CAN_MANAGE'."}

In [0]:
for underlying in underlyings:
    table_name = "zetadex_feature_store.agg_trades_24h_rolling_" + underlying.lower()
    print(f"Underlying: {underlying}")
    print(f"Table Name: {table_name}")

    # On the fly transformation to get the last 24hrs volume from our 1h agg'ed tables
    agg_trades_24h_rolling_df_underlying = \
        (spark.table("zetadex_mainnet.agg_trades_market_1h")
         .filter(F.col("timestamp_window.start") >= (F.date_trunc("hour", F.current_timestamp()) - F.expr('INTERVAL 24 HOUR')))
         .filter(F.col("underlying") == underlying)
         .agg(
             F.sum("trades_count").alias("trades_count"),
             F.sum("volume").alias("volume"),
             F.sum("notional_volume").alias("notional_volume"),
             F.sum("premium_sum").alias("premium_sum"),
          )
          .withColumn("timestamp", F.date_trunc("hour", F.current_timestamp()))
          .withColumn("date_", F.to_date("timestamp"))
          .withColumn("hour_", F.date_format("timestamp", "HH").cast("int"))
         )
    agg_trades_24h_rolling_df_underlying.show()

    try:
        result = fs.get_table(table_name)
        print(result)
        print('Table Already Exists...')
    except ValueError:
        print('Creating New Table...')
        fs.create_table(
            name=table_name,
            primary_keys=["timestamp"],
            df=agg_trades_24h_rolling_df_underlying,
            partition_columns="date_",
            description=f"Rolling 24hr trade summary metrics {underlying}",
        )
    except Exception:
        print('Table Already Exists...')
        pass

    # Write new results to table
    fs.write_table(
      name=table_name,
      df=agg_trades_24h_rolling_df_underlying,
      mode="merge",
    )

    fs.publish_table(
      name=table_name,
      online_store=online_store,
    #   filter_condition=f"date_ = '{current_date}' and hour_ = '{current_hour}'",
      features=[
        'trades_count',
        'volume',
        'notional_volume',
        'premium_sum',
        'timestamp'],
      mode='merge'
    )

In [0]:
# On the fly transformation forming unique primary key for ddb using a concatenation of market values
agg_prices_market_1h_df = \
    (spark.table("zetadex_mainnet.agg_prices_market_1h")
     .filter(F.col("timestamp") >= (F.date_trunc("hour", F.current_timestamp()) - F.expr('INTERVAL 1 HOUR')))
     .withColumn("ddb_key", F.concat(F.col("underlying"), F.lit("#"), F.unix_timestamp(F.col("expiry_timestamp")), F.lit("#"), F.col("kind"), F.lit("#"), F.col("strike")))
     )
agg_prices_market_1h_df = agg_prices_market_1h_df.drop("timestamp_window", "underlying", "strike", "kind", "date_", "hour_")
agg_prices_market_1h_df.show()

# fs.create_table(
#     name='zetadex_feature_store.agg_prices_market_1h_v3',
#     primary_keys="ddb_key",
#     df=agg_prices_market_1h_df,
#     description="Aggregated options markets 1h",
# )


# Write new results to table
fs.write_table(
  name='zetadex_feature_store.agg_prices_market_1h_v3',
  df=agg_prices_market_1h_df,
  mode="merge",
)

fs.publish_table(
  name='zetadex_feature_store.agg_prices_market_1h_v3',
  online_store=online_store,
#   filter_condition=f"date_ = '{current_date}' and hour_ = '{current_hour}'",
  features=[
      'ddb_key',
      'open_interest',
      'open_interest_usd',
      'theo',
      'delta',
      'vega',
      'sigma',
      'timestamp',
      'expiry_timestamp'],
  mode='merge'
)

+-------------------+------------------+--------------+-------------------+------------------+------------------+------------------+-------------------+--------------------+
 expiry_timestamp| theo| delta| vega| sigma| open_interest| open_interest_usd| timestamp| ddb_key|
+-------------------+------------------+--------------+-------------------+------------------+------------------+------------------+-------------------+--------------------+
2022-07-15 08:00:00| 37.390224| null| null| null| 1584.3895| 59240.678308248|2022-07-10 06:00:00|SOL#1657872000#fu...|
2022-07-15 08:00:00| 5.844058|0.141451682124| 0.992964715481758| 1.319780147705414| 0.0| 0.0|2022-07-10 06:00:00|SOL#1657872000#ca...|
2022-07-22 08:00:00| 16.875611|0.934262141661| 0.8707022133196782|1.2469914923056808| 0.0| 0.0|2022-07-10 06:00:00|SOL#1658476800#pu...|
2022-07-22 08:00:00| 2.423116|0.389527018145| 2.6108741691246142| 1.145905694395034| 0.0| 0.0|2022-07-10 06:00:00|SOL#1658476800#pu...|
2022-07-22 08:00:00| 6.097869| 0.67916710099| 2.4371144663185778|1.1324323434697248| 0.0| 0.0|2022-07-10 06:00:00|SOL#1658476800#pu...|
2022-07-22 08:00:00| 3.77186|0.389527018145| 2.6108741691246142| 1.145905694395034| 0.0| 0.0|2022-07-10 06:00:00|SOL#1658476800#ca...|
2022-07-22 08:00:00| 22.764443|0.965818987353| 0.5158782099675443|1.3373918473055673| 0.0| 0.0|2022-07-10 06:00:00|SOL#1658476800#pu...|
2022-07-22 08:00:00| 37.348744| null| null| null| 0.001| 0.037348744|2022-07-10 06:00:00|SOL#1658476800#fu...|
2022-07-22 08:00:00|1.2588460000000001|0.220169170154| 2.016367763172863|1.2979310136233704| 1000.0|1258.8460000000002|2022-07-10 06:00:00|SOL#1658476800#pu...|
2022-07-15 08:00:00|11.540704999999999|0.040888612594| 0.3887112452014539| 1.88249269945947| 199.2675| 2299.6874335875|2022-07-10 06:00:00|SOL#1657872000#ca...|
2022-07-22 08:00:00| 0.300525|0.047515898125| 0.6740163020451179| 1.946602588404| 465.0| 139.744125|2022-07-10 06:00:00|SOL#1658476800#pu...|
2022-07-22 08:00:00| 2.712542|0.493947479927| 2.7153180585306504|1.1066977338936235| 4.0| 10.850168|2022-07-10 06:00:00|SOL#1658476800#ca...|
2022-07-15 08:00:00| 1.326272|0.362533880384| 1.661270625356304|1.1209036379947535| 1010.0|1339.5347199999999|2022-07-10 06:00:00|SOL#1657872000#pu...|
2022-07-15 08:00:00| 0.032687|0.009933188799|0.11740248677389703| 2.026602588404|3235.7155000000002|105.76583254850001|2022-07-10 06:00:00|SOL#1657872000#pu...|
2022-07-15 08:00:00| 0.453834|0.141451682124| 0.992964715481758| 1.319780147705414| 1454.0| 659.874636|2022-07-10 06:00:00|SOL#1657872000#pu...|
2022-07-15 08:00:00| 0.782339|0.231913722666| 1.3515769068498038|1.2144925837422083| 5742.6335| 4492.6861497565|2022-07-10 06:00:00|SOL#1657872000#pu...|
2022-07-15 08:00:00| 0.070393|0.963165689144| 0.3569740536953417|1.1355432077561547| 11460.911| 806.767908023|2022-07-10 06:00:00|SOL#1657872000#ca...|
2022-07-22 08:00:00| 1.446613| 0.67916710099| 2.4371144663185778|1.1324323434697248| 1000.0|1446.6129999999998|2022-07-10 06:00:00|SOL#1658476800#ca...|
2022-07-15 08:00:00| 2.220826|0.525784412094| 1.7637210365478189|1.0671048225735817| 1000.5|2221.9364130000004|2022-07-10 06:00:00|SOL#1657872000#pu...|
2022-07-15 08:00:00| 21.390572| 1.2580235E-4| 0.0| 2.026602588404| 96.589| 2066.093958908|2022-07-10 06:00:00|SOL#1657872000#ca...|
+-------------------+------------------+--------------+-------------------+------------------+------------------+------------------+-------------------+--------------------+
only showing top 20 rows

--------------------------------------------------------------------------- 
 Exception Traceback (most recent call last)
 <command-1646467190265717> in <module> 
 17 
 18 # Write new results to table 
 ---> 19 fs.write_table(
 20 name = 'zetadex_feature_store.agg_prices_market_1h_v3' , 
 21 df = agg_prices_market_1h_df , 

 /databricks/.python_edge_libs/databricks/feature_store/client.py in write_table (self, name, df, mode, checkpoint_location, trigger) 
 1563 : return : If ` ` df . isStreaming ` ` , returns a PySpark : class : ` StreamingQuery < pyspark . sql . streaming . StreamingQuery > ` . : obj : ` None ` otherwise . 
 1564 """
 -> 1565 return self._write_table(
 1566 name = name , 
 1567 df = df , 

 /databricks/.python_edge_libs/databricks/feature_store/client.py in _write_table (self, name, df, mode, req_context, checkpoint_location, trigger, producer_action) 
 1362 FeatureStoreClient . _check_schema_top_level_types_supported ( df . schema ) 
 1363 self . _check_feature_table_exists_in_hive ( name ) 
 -> 1364 feature_table = self . _catalog_client . get_feature_table ( name , req_context ) 
 1365 
 1366 # We know from the successful `get_feature_table call` above that the user has 

 /databricks/.python_edge_libs/databricks/feature_store/catalog_client.py in get_feature_table (self, feature_table, req_context) 
 157 
 158 def get_feature_table ( self , feature_table : str , req_context : RequestContext ) : 
 --> 159 response_proto = self . _get_feature_table ( feature_table , req_context ) 
 160 return FeatureTable . from_proto ( response_proto . feature_table ) 
 161 

 /databricks/.python_edge_libs/databricks/feature_store/catalog_client.py in _get_feature_table (self, feature_table, req_context) 
 132 def _get_feature_table ( self , feature_table : str , req_context : RequestContext ) : 
 133 req_body = GetFeatureTable ( name = feature_table ) 
 --> 134 return self . _call_endpoint ( GetFeatureTable , req_body , req_context ) 
 135 
 136 # CRUD API to call Feature Catalog 

 /databricks/.python_edge_libs/databricks/feature_store/catalog_client.py in _call_endpoint (self, api, json_body, req_context) 
 121 endpoint , method = _METHOD_TO_INFO [ api ] 
 122 response_proto = api . Response ( ) 
 --> 123 return call_endpoint(
 124 self . _get_host_creds ( ) , 
 125 endpoint , 

 /databricks/.python_edge_libs/databricks/feature_store/utils/rest_utils.py in call_endpoint (host_creds, endpoint, method, json_body, response_proto, req_context) 
 199 extra_headers = req_context . get_headers ( ) , 
 200 )
 --> 201 response = verify_rest_response ( response , endpoint ) 
 202 js_dict = json . loads ( response . text ) 
 203 json_to_proto ( js_dict = js_dict , message = response_proto ) 

 /databricks/.python_edge_libs/databricks/feature_store/utils/rest_utils.py in verify_rest_response (response, endpoint) 
 118 if _can_parse_as_json ( response . text ) : 
 119 # ToDo(ML-20622): return cleaner error to client, eg: mlflow.exceptions.RestException 
 --> 120 raise Exception ( json . loads ( response . text ) ) 
 121 else : 
 122 base_msg = (

 Exception : {'error_code': 'PERMISSION_DENIED', 'message': "Request failed access control checks. User has no permissions on 'zetadex_feature_store.agg_prices_market_1h_v3' but requires one of 'CAN_VIEW_METADATA', 'CAN_EDIT_METADATA', 'CAN_MANAGE'."}

In [0]:
row1 = (spark.table("zetadex_mainnet.agg_prices_market_1h").agg({"timestamp": "max"}).collect())[0]
print(row1["max(timestamp)"])
agg_prices_market_total_df = \
(spark.table("zetadex_mainnet.agg_prices_market_1h").filter(F.col("timestamp") == row1["max(timestamp)"]).groupBy().sum("open_interest", "open_interest_usd").drop("sum(strike)", "sum(theo)", "sum(delta)", "sum(vega)", "sum(sigma)", "sum(hour_)").withColumn("timestamp", F.lit(row1["max(timestamp)"])).withColumnRenamed("sum(open_interest)", "total_open_interest").withColumnRenamed("sum(open_interest_usd)", "total_open_interest_usd"))

agg_prices_market_total_puts = \
(spark.table("zetadex_mainnet.agg_prices_market_1h").filter(F.col("timestamp") == row1["max(timestamp)"]).filter(F.col("kind") == "put").groupBy().sum("open_interest", "open_interest_usd").drop("sum(strike)", "sum(theo)", "sum(delta)", "sum(vega)", "sum(sigma)", "sum(hour_)").withColumn("timestamp", F.lit(row1["max(timestamp)"])).withColumnRenamed("sum(open_interest)", "total_open_interest").withColumnRenamed("sum(open_interest_usd)", "total_open_interest_usd"))

agg_prices_market_total_calls = \
(spark.table("zetadex_mainnet.agg_prices_market_1h").filter(F.col("timestamp") == row1["max(timestamp)"]).filter(F.col("kind") == "call").groupBy().sum("open_interest", "open_interest_usd").drop("sum(strike)", "sum(theo)", "sum(delta)", "sum(vega)", "sum(sigma)", "sum(hour_)").withColumn("timestamp", F.lit(row1["max(timestamp)"])).withColumnRenamed("sum(open_interest)", "total_open_interest").withColumnRenamed("sum(open_interest_usd)", "total_open_interest_usd"))

put_call_ratio = agg_prices_market_total_puts.collect()[0]['total_open_interest'] / agg_prices_market_total_calls.collect()[0]['total_open_interest']
print(put_call_ratio)
# agg_prices_market_total_puts.show()
# agg_prices_market_total_calls.show()

agg_prices_market_total_df = agg_prices_market_total_df.withColumn("put_call_ratio", F.lit(put_call_ratio))
agg_prices_market_total_df.show()

fs.create_table(
    name='zetadex_feature_store.agg_prices_market_oi',
    primary_keys="timestamp",
    df=agg_prices_market_total_df,
    description="Aggregated total open interest",
)

# Write new results to table
fs.write_table(
  name='zetadex_feature_store.agg_prices_market_oi',
  df=agg_prices_market_total_df,
  mode="merge",
)

fs.publish_table(
  name='zetadex_feature_store.agg_prices_market_oi',
  online_store=online_store,
  mode='merge'
)

2022-07-10 06:00:00
1.2473557168954226
+-------------------+-----------------------+-------------------+------------------+
total_open_interest|total_open_interest_usd| timestamp| put_call_ratio|
+-------------------+-----------------------+-------------------+------------------+
 68086.86550000001| 109198.57086473198|2022-07-10 06:00:00|1.2473557168954226|
+-------------------+-----------------------+-------------------+------------------+

In [0]:
for underlying in underlyings:
    table_name = 'zetadex_feature_store.agg_prices_market_oi_' + underlying.lower()
    print(f"Underlying: {underlying}")
    print(f"Table Name: {table_name}")

    row1 = (spark.table("zetadex_mainnet.agg_prices_market_1h").agg({"timestamp": "max"}).collect())[0]
    print(row1["max(timestamp)"])

    agg_prices_market_total_df_underlying = \
    (spark.table("zetadex_mainnet.agg_prices_market_1h").filter(F.col("timestamp") == row1["max(timestamp)"]).filter(F.col("underlying") == underlying).groupBy().sum("open_interest", "open_interest_usd").drop("sum(strike)", "sum(theo)", "sum(delta)", "sum(vega)", "sum(sigma)", "sum(hour_)").withColumn("timestamp", F.lit(row1["max(timestamp)"])).withColumnRenamed("sum(open_interest)", "total_open_interest").withColumnRenamed("sum(open_interest_usd)", "total_open_interest_usd"))

    agg_prices_market_total_puts_underlying = \
    (spark.table("zetadex_mainnet.agg_prices_market_1h").filter(F.col("timestamp") == row1["max(timestamp)"]).filter(F.col("kind") == "put").filter(F.col("underlying") == underlying).groupBy().sum("open_interest", "open_interest_usd").drop("sum(strike)", "sum(theo)", "sum(delta)", "sum(vega)", "sum(sigma)", "sum(hour_)").withColumn("timestamp", F.lit(row1["max(timestamp)"])).withColumnRenamed("sum(open_interest)", "total_open_interest").withColumnRenamed("sum(open_interest_usd)", "total_open_interest_usd"))

    agg_prices_market_total_calls_underlying = \
    (spark.table("zetadex_mainnet.agg_prices_market_1h").filter(F.col("timestamp") == row1["max(timestamp)"]).filter(F.col("kind") == "call").filter(F.col("underlying") == underlying).groupBy().sum("open_interest", "open_interest_usd").drop("sum(strike)", "sum(theo)", "sum(delta)", "sum(vega)", "sum(sigma)", "sum(hour_)").withColumn("timestamp", F.lit(row1["max(timestamp)"])).withColumnRenamed("sum(open_interest)", "total_open_interest").withColumnRenamed("sum(open_interest_usd)", "total_open_interest_usd"))

    try:
        put_call_ratio_underlying = agg_prices_market_total_puts_underlying.collect()[0]['total_open_interest'] / agg_prices_market_total_calls_underlying.collect()[0]['total_open_interest']
    except Exception:
        put_call_ratio_underlying = 0
    print(put_call_ratio_underlying)

    agg_prices_market_total_df_underlying = agg_prices_market_total_df_underlying.withColumn("put_call_ratio", F.lit(put_call_ratio_underlying))
    agg_prices_market_total_df_underlying.show()

    try:
        result = fs.get_table(table_name)
        print(result)
        print('Table Already Exists...')
    except ValueError:
        print('Creating New Table...')
        fs.create_table(
            name=table_name,
            primary_keys="timestamp",
            df=agg_prices_market_total_df_underlying,
            description=f"Aggregated total open interest {underlying}",
        )
    except Exception:
        print('Table Already Exists...')

    # Write new results to table
    fs.write_table(
      name=table_name,
      df=agg_prices_market_total_df_underlying,
      mode="merge",
    )

    fs.publish_table(
      name=table_name,
      online_store=online_store,
      mode='merge'
    )